# Data Challenge - Πρόβλεψη Πληρότητας Πτήσεων

Στα πλαίσια της εργασίας του μαθήματος "Εξόρυξη Γνώσης από Βάσεις Δεδομένων και τον Παγκόσμιο Ιστό", θα δουλέψετε πάνω σε ένα πρόβλημα κατηγοριοποίησης. Συγκεκριμένα, σας δίνεται ένα σύνολο δεδομένων το οποίο αποτελείται από μερικές χιλιάδες πτήσεις, όπου κάθε πτήση περιγράφεται απο ένα σύνολο μεταβλητών (αεροδρόμιο αναχώρησης, αεροδρόμιο άφιξης, κτλ). Κάθε πτήση χαρακτηρίζεται επίσης από μια μεταβλητή που σχετίζεται με τον αριθμό των επιβατών της πτήσης (π.χ. κάθε τιμή της μεταβλητής σχετίζεται με ενα εύρος πλήθους επιβατών). Για κάποιες πτήσεις, η τιμή της μεταβλητής  είναι γνωστή, ενώ για άλλες όχι. Στόχος σας είναι να προβλέψετε την τιμή της μεταβλητής για τις πτήσεις για τις οποίες δεν είναι διαθέσιμη.

### Σύνολο Δεδομένων

Το αρχείο με όνομα `train.csv` περιέχει τα δεδομένα εκπαίδευσης (training set) του προβλήματος, ενώ το αρχείο `test.csv` περιέχει τα δεδομένα ελέγχου (test set) του προβλήματος. Κάθε γραμμή των δυο αυτών αρχείων αντιστοιχεί σε μια πτήση η οποία χαρακτηρίζεται από τις εξής μεταβλητές:

Μεταβλητή | Περιγραφή
--- | --- 
DateOfDeparture | Ημερομηνία αναχώρησης
Departure | Κωδικός αεροδρομίου αναχώρησης
CityDeparture | Πόλη αναχώρησης
LongitudeDeparture 	 | Γεωγραφικό μήκος αεροδρομίου αναχώρησης
LatitudeDeparture 	 | Γεωγραφικό πλάτος αεροδρομίου αναχώρησης
Arrival | Κωδικός αεροδρομίου άφιξης
CityArrival | Πόλη άφιξης
LongitudeArrival | Γεωγραφικό μήκος αεροδρομίου άφιξης
LatitudeArrival | Γεωγραφικό πλάτος αεροδρομίου άφιξης
WeeksToDeparture | Πόσες εβδομάδες πριν την αναχώρηση της πτήσης κατά μέσο όρο έκλεισαν οι επιβάτες τα εισητήριά τους
std_wtd | Τυπική απόκλιση για το παραπάνω 
Το training set περιέχει μια επιπλέον μεταβλητή (`PAX`) η οποία έχει σχέση με τον αριθμό των επιβατών της πτήσης. Η μεταβλητή αυτή παίρνει 8 διαφορετικές τιμές (τιμές από 0 έως 7 οπότε 8 κατηγορίες συνολικά). Κάθε κατηγορία υποδηλώνει πόσοι περίπου επιβάτες χρησιμοποίησαν την πτήση. Οι αριθμοί στις κατηγορίες έχουν ανατεθεί με τυχαίο τρόπο, οπότε μην θεωρήσετε ότι η κατηγορία 0 υποδηλώνει πολύ λίγους επιβάτες ενώ η κατηγορία 7 πάρα πολλούς επιβάτες. Η μεταβλητή `PAX` λείπει από το test set καθώς πρόκειται για την μεταβλητή που πρέπει να προβλέψετε στα πλαίσια της παρούσας εργασίας.

Παρακάτω σας δίνεται κώδικας ο οποίος φορτώνει τα δεδομένα εκπαίδευσης σε ένα DataFrame της βιβλιοθήκης Pandas και τυπώνει τις πρώτες 5 γραμμές. Οπότε μπορείτε να δείτε τις 12 μεταβλητές του προβλήματος.

In [1]:
import pandas as pd

df_train = pd.read_csv('ntrain.csv')
df_train.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,Distance,PAX
0,2011-12-05,ORD,Chicago,41.978603,-87.904842,EWR,Newark,40.692500,-74.168667,8.352941,5.667243,1527.939887,7
1,2013-03-01,ATL,Atlanta,33.636719,-84.428067,LGA,New York,40.777245,-73.872608,10.421053,10.001754,1181.336943,7
2,2012-02-06,ORD,Chicago,41.978603,-87.904842,BOS,Boston,42.364347,-71.005181,9.250000,7.136821,1879.752387,7
3,2012-12-10,ORD,Chicago,41.978603,-87.904842,SFO,San Francisco,37.618972,-122.374889,8.666667,7.404291,3833.459735,7
4,2012-04-13,SFO,San Francisco,37.618972,-122.374889,JFK,New York,40.639751,-73.778925,14.037037,9.858544,5403.554833,7


Επίσης, όσον αφορά τις διαστάσεις του training set, όπως βλέπετε παρακάτω, αποτελείται από 8899 γραμμές και 12 στήλες.

In [2]:
df_train.shape

(8899, 13)

In [ ]:
#from math import sin, cos, sqrt, atan2, radians

#def distance_calculation(lat1,lon1,lat2,lon2): 
    
#R = 6373.0 # approximate radius of earth in km

#lat1 = radians(lat1)
#lon1 = radians(lon1)
#lat2 = radians(lat2)
#lon2 = radians(lon2)

#dlon = lon2 - lon1
#dlat = lat2 - lat1

#a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
#c = 2 * atan2(sqrt(a), sqrt(1 - a))

#distance = R * c

#return distance

In [ ]:
#paradeigma ektelesis methodou

#print(distance_calculation(-87.904842,41.978603,-74.168667,40.692500))


In [ ]:
#Ftiaxnei ta dunamosunola
def powerset(seq):
    #counter = 0
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item
            #couter += 1
    #print(counter)


In [ ]:
l = [0, 1, 2, 3, 4, 5, 6, 7]
r = [x for x in powerset(l)]
r.sort()
print(r)
print(r[0])

In [ ]:
#import csv

#def powerset(s):
    #with open('regression_results', 'w', newline='') as csvfile:
        #writer = csv.writer(csvfile, delimiter=',')
        #writer.writerow(['Set', 'Result'])
    #x = len(s)
    #for i in range(1 << x):
        #print(i)
        #print([s[j] for j in range(x) if (i & (1 << j))])

In [ ]:
#powerset([0,1,2,3,4,5,6,7]) paizei
print(r.sort())
print(r)

In [ ]:
import numpy as np

def whoTodrop(i):
    columns=[];
    for k in range(8):
           for j in range(len(r[i])):
                if(k!=r[i][j]):
                    columns.append(k)
    return columns 

In [ ]:
df_test = pd.read_csv('ntest.csv')
y_train = df_train[['PAX']]
from sklearn.model_selection import train_test_split
#df_train, df_test, y_train, y_test = train_test_split(df_train, y_train, test_size=0.2, random_state=42)
#y_train = np.ravel(y_train)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df_train['Departure'])
df_train['Departure'] = le.transform(df_train['Departure'])
df_train['Arrival'] = le.transform(df_train['Arrival'])
df_test['Departure'] = le.transform(df_test['Departure'])
df_test['Arrival'] = le.transform(df_test['Arrival'])
le.fit(df_train['CityDeparture'])
df_train['CityDeparture'] = le.transform(df_train['CityDeparture'])
df_train['CityArrival'] = le.transform(df_train['CityArrival'])
df_test['CityDeparture'] = le.transform(df_test['CityDeparture'])
df_test['CityArrival'] = le.transform(df_test['CityArrival'])
le = LabelEncoder()
le.fit(df_train['DateOfDeparture'])
df_train['DateOfDeparture'] = le.transform(df_train['DateOfDeparture'])
df_test['DateOfDeparture'] = le.transform(df_test['DateOfDeparture'])
print(df_train)

In [ ]:
#TEST
df_train.columns.get_loc('Departure')

In [ ]:


def findCateg(i):
    catf =[];
    t0 = 0
    t1 = 0
    t2 = 0
    t3 = 0
    t4 = 0
    for k in range(len(r[i])):
        if(r[i][k]==1):
            t1 = 1
        elif(r[i][k]==2):
            t2 = 1
        elif(r[i][k]==3):
            t3 = 1
        elif(r[i][k]==4):
            t4 = 1
        elif(r[i][k]==0):
            t0 = 1
    if(t0==0):
        ind0=dfTr.columns.get_loc('DateOfDeparture')
        catf.append(ind0)
    if(t1==0):
        ind0=dfTr.columns.get_loc('Departure')
        catf.append(ind0)
    if(t2==0):
        ind0=dfTr.columns.get_loc('CityDeparture')
        catf.append(ind0)
    if(t3==0):
        ind0=dfTr.columns.get_loc('Arrival')
        catf.append(ind0)
    if(t4==0):
        ind0=dfTr.columns.get_loc('CityArrival')
        catf.append(ind0)
    return catf

In [ ]:
#NEW ENTRY
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import f1_score

df_train.drop(df_train.columns[[3,4,7,8,12]], axis=1, inplace=True)
df_test.drop(df_test.columns[[3,4,7,8,12]], axis=1, inplace=True)
print(df_train.columns[3])
score=[]
r.sort()
for i in range(len(r)):
    if(len(r[i])<8):
        dfTr = df_train.copy()
        dfTe= df_test.copy()
        col=whoTodrop(i)
        #print(dfTr)
        #print(df_train)
        #print(col)
        print(r[i])
        dfTr.drop(dfTr.columns[r[i]], axis=1, inplace=True)
        print(dfTr)
        print("DF_TRAIN")
        print(df_train)
        dfTe.drop(dfTe.columns[r[i]], axis=1, inplace=True)
        cat_f = findCateg(i)
        if cat_f:
            print('in onehot')
            enc = OneHotEncoder(categorical_features=cat_f,sparse=False)#check argument
            enc.fit(dfTr)
            X_train = enc.transform(dfTr)
            X_test = enc.transform(dfTe)
        else:
            X_train = dfTr
            X_test = dfTe

        #X_train = dfTr
        #X_test = dfTe
        #print(X_test)
        clf = LogisticRegression()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        scr=f1_score(y_test, y_pred, average='micro')
        print(i,scr)
        score.append(scr)
    elif(i==0):
        dfTr = df_train.copy()
        dfTe= df_test.copy()
        #col=whoTodrop(i)
        cat_f = [0,1,2,3,4]
        enc = OneHotEncoder(categorical_features=cat_f,sparse=False)#check argument
        enc.fit(dfTr)
        X_train = enc.transform(dfTr)
        X_test = enc.transform(dfTe)        
        #X_train = dfTr
        #X_test = dfTe
        #print(X_test)
        clf = LogisticRegression()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        scr=f1_score(y_test, y_pred, average='micro')
        print(i,scr)
        score.append(scr)
    elif(len(r[i])==8):
        score.append(0)



In [ ]:
print(len(r))

In [ ]:
import csv
def calculate_dynamic_set(num_columns):
    
    with open('regression_results', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Set to drop', 'Result'])
        for i in range(len(r)):
            writer.writerow([r[i], score[i]])
    

In [ ]:
calculate_dynamic_set(8)

In [ ]:
#for i in range(len(df_train)):
#a = distance_calculation(df_train.iat[0,4],df_train.iat[0,3],df_train.iat[0,8],df_train.iat[0,7])
#print(a)

#print(len(df_train))
#print(df_train['Departure'][0])


#df_train['Distance'] = 0.0
#for i in range(len(df_train)):
#df_train['Distance'][i] = distance_calculation(df_train['LatitudeDeparture'][i],df_train['LongitudeDeparture'][i],df_train['LatitudeArrival'][i],df_train['LongitudeArrival'][i])

#print(df_train)


In [ ]:
#df_train.head()

Δηλαδή έχετε διαθέσιμα τα δεδομένα απο 8899 πτήσεις για τις οποίες ξέρετε την κατηγορία του αριθμού επιβάτών `PAX`.

Αντίθετα, το αρχείο με όνομα `test.csv` περιέχει το test set του προβλήματος. Όπως και στην περίπτωση του training set, σας δίνεται και τώρα κώδικας ο οποίος φορτώνει το test set σε ένα DataFrame της βιβλιοθήκης Pandas και τυπώνει τις πρώτες 5 γραμμές του.

In [3]:
df_test = pd.read_csv('ntest.csv')
#df_test.head()

#df_test['Distance'] = 0.0
#for i in range(len(df_test)):
#df_test['Distance'][i] = distance_calculation(df_test['LatitudeDeparture'][i],df_test['LongitudeDeparture'][i],df_test['LatitudeArrival'][i],df_test['LongitudeArrival'][i])
    

In [4]:
df_test.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,Distance
0,2012-10-21,DFW,Dallas-Fort Worth,32.896828,-97.037997,SFO,San Francisco,37.618972,-122.374889,14.600000,11.575837,2821.532925
1,2012-09-13,LAX,Los Angeles,33.942536,-118.408075,ATL,Atlanta,33.636719,-84.428067,14.730769,13.364304,3779.583120
2,2012-09-04,ORD,Chicago,41.978603,-87.904842,IAH,Houston,29.984433,-95.341442,8.470588,5.885551,823.505267
3,2012-08-13,DEN,Denver,39.861656,-104.673178,PHX,Phoenix,33.434278,-112.011583,8.200000,6.292853,845.501646
4,2012-09-10,ORD,Chicago,41.978603,-87.904842,SEA,Seattle,47.449000,-122.309306,12.090909,9.138662,3825.798893


Όσον αφορά τις διαστάσεις του test set, όπως βλέπετε παρακάτω, αποτελείται από 2229 γραμμές και 11 στήλες. Η στήλη που λείπει αφορά τη μεταβλητή `PAX` την οποία στόχος σας είναι να προβλέψετε.

In [5]:
df_test.shape

(2229, 12)

### Στόχος

Όπως αναφέρθηκε και παραπάνω, στόχος σας είναι να προβλέψετε την κατηγορία αριθμού επιβατών (μεταβλητή `PAX`) για κάθε πτήση του test set. Πρόκειται για ένα supervised learning πρόβλημα. Θα πρέπει να επεξεργαστείτε τα δεδομένα του training set και στη συνέχεια, θα τα χρησιμοποιήσετε για να εκπαιδεύσετε κάποιο ταξινομητή, τον οποίο θα χρησιμοποιήσετε για να κάνετε προβλέψεις σχετικά με τα δεδομένα του test set.


### Αξιολόγηση

Η αξιολόγηση των προβλέψεων θα γίνει με βάση το `micro F1-score`. Το F1-score παίρνει τιμές μεταξύ 0 και 1. Όσο μεγαλύτερη η τιμή του F1-score, τόσο καλύτερος ο ταξινομητής που παρήγαγε τις προβλέψεις. Για κάθε μια από τις 8 κατηγορίες του test set μας, ο ταξινομητής θα κάνει κάποιες σωστές προβλέψεις και κάποιες λάθος. Πιο συγκεκριμένα, για μια κατηγορία $c$:
- τα παραδείγματα που ανήκουν στην κατηγορία $c$ και ο ταξινομητής προέβλεψε ότι πράγματι ανήκουν στην κατηγορία $c$ ονομάζονται True Positives (TP).
- τα παραδείγματα που ανήκουν στην κατηγορία $c$ και ο ταξινομητής προέβλεψε λανθασμένως ότι ανήκουν σε κάποια άλλη κατηγορία ονομάζονται False Positives (FP).
- τα παραδείγματα που ανήκουν σε κάποια κατηγορία διαφορετική της $c$ και ο ταξινομητής προέβλεψε ότι πράγματι ανήκουν σε κάποια κατηγορία διαφορετική της $c$ ονομάζονται True Negatives (TN).
- τα παραδείγματα που ανήκουν σε κατηγορία διαφορετική της $c$ και ο ταξινομητής προέβλεψε ότι ανήκουν στην κατηγορία $c$ ονομάζονται False Negatives (FN).

Δεδομένων των παραπάνω ορισμών, μπορούμε συνήθως να υπολογίσουμε μερικές μετρικές απόδοσης του ταξινομητή μας, λαμβάνοντας υπόψη τον συνολικό (για όλες τις κατηγορίες) αριθμό True Positives (TP), False Positives (FP) και False Negatives (FN).
Στο πρόβλημα που σας έχει ανατεθεί υπάρχουν συνολικά 8 κατηγορίες $c_1, c_2, \ldots, c_8$ οπότε μπορούμε να υπολογίσουμε:
Πρώτα, το `Recall` το οποίο ορίζεται ως:
$$ Recall = \frac{\sum_{i=1}^8 TP_{c_i}}{\sum_{i=1}^8TP_{c_i}+\sum_{i=1}^8FN_{c_i}} $$
και μετράει συνολικά το ποσοστό των παραδειγμάτων που προβλέφτηκαν ότι ανήκουν στην κατηγορία $c_i$ σε σχέση με αυτά που ανήκουν πράγματι στην κατηγορία $c_i$. Έπειτα, το `Precision` το οποίο μας λέει πόσα από τα παραδείγματα που προβλέφτηκαν ότι ανήκουν στην κατηγορία $c_i$, ανήκουν πράγματι σε αυτή την κατηγορία, και ορίζεται ως εξης:
$$ Precision = \frac{\sum_{i=1}^8 TP_{c_i}}{\sum_{i=1}^8TP_{c_i}+\sum_{i=1}^8FP_{c_i}} $$

Το `micro F1-score` ειναι ο αρμονικός μέσος του Precision και του Recall, και ορίζεται ως εξής:
$$ micro\text{ } F1-score =  \frac{2*Recall*Precision}{Recall + Precision} $$
Πρόκειται για μια ευρέως διαδεδομένη μετρική απόδοσης η οποία είναι και αξιόπιστη δεδομένου ότι λαμβάνει υπόψη τόσο τα False Positives όσο και τα False Negatives.

Αφότου υποβάλετε μια λύση στην πλατφόρμα (όπως περιγράφεται παρακάτω), το micro F1-score της λύσης σας υπολογίζεται αυτόματα και εμφανίζεται στην οθόνη.

### Υποβολή Λύσης

Αφότου έχετε σχεδιάσει και τρέξει τον αλγοριθμό σας, και έχετε προβλέψει την κατηγορία επιβατών για κάθε πτήση του test set, πρέπει να υποβάλετε την λύση σας στην πλατφόρμα Kaggle ώστε να αξιολογηθεί. Συγκεκριμένα, στην κεντρική οθόνη υπάρχει η επιλογή `Submit Predictions` η οποία σας δίνει τη δυνατότητα να ανεβάσετε τις προβλέψεις σας. Οι προβλέψεις σας θα πρέπει να συμπεριληφθούν σε ένα αρχείο όπου κάθε γραμμή περιέχει την πρόβλεψή σας για την πτήση η οποία βρίσκεται στην ίδια γραμμή του test set. Για παράδειγμα, παρακάτω σας δίνονται οι τρείς πρώτες πτήσεις του test set:
``` html
2012-10-21,DFW,Dallas-Fort Worth,32.896828,-97.037997,SFO,San Francisco,37.618972,-122.374889,14.6,11.5758369028
2012-09-13,LAX,Los Angeles,33.942536,-118.408075,ATL,Atlanta,33.636719,-84.428067,14.7307692308,13.3643037748
2012-09-04,ORD,Chicago,41.978603,-87.904842,IAH,Houston,29.984433,-95.341442,8.47058823529,5.88555060146
```
Η πρώτη γραμμή του αρχείου που θα υποβάλετε θα πρέπει να περιέχει την πρόβλεψή σας για την κατηγορία αριθμού επιβατών της πρώτης πτήσης, η δεύτερη γραμμή θα πρέπει να περιέχει την πρόβλεψή σας για την δεύτερη πτήση και αντίστοιχα η τρίτη γραμμή θα πρέπει να περιέχει την πρόβλεψή σας για την τρίτη πτήση.

**Προσοχή**: Στο αρχείο που θα υποβάλετε, κάθε γραμμή θα πρέπει να περιέχει 2 αριθμούς, έναν για το `Id` της πτήσης του test set και έναν για το `Label` που υποδηλώνει την κατηγορία επιβατών (αριθμοί 0-7). Επιπλέον, θα πρέπει να υπάρχουν συνολικά **2229** γραμμές όσες και οι πτήσεις για τις οποίες πρέπει να κάνετε πρόβλεψη. Τέλος, η κωδικοποίηση του αρχείου θα πρέπει να είναι **utf-8**. Φυσικά θα υπάρχει και μία ακόμη γραμμή στην αρχή που περιέχει τό όνομα των πεδίων `Id` και `Label`.

### Αρχικός Κώδικας Python

Σας δίνεται ένας αρχικός Python κώδικας ο οποίος χρησιμοποιεί ως μόνη πληροφορία το αεροδρόμιο από το οποίο αναχώρησε η πτήση και το αεροδρόμιο στο οποίο προσγειώθηκε για να προβλέψει την κατηγορία αριθμού επιβατών. Συγκεκριμένα, ο παρακάτω κώδικας αποθηκεύει στη μεταβλητή `y_train` τη στήλη `PAX` του training set, δηλαδή την στήλη την οποία θέλουμε να μάθουμε να προβλέπουμε. Στη συνέχεια, διαγράφει όλες τις στήλες του training set εκτός από τις `Departure` και `Arrival` οι οποίες αντιστοιχούν στους κωδικούς αεροδρομίων αναχώρησης και άφιξης. Επίσης για να είναι πιό εύκολος ο έλεγχος της λύσης σας (τοπικά), μπορείτε αν θέλετε να παράγετε ένα νέο training και test set απο το αρχικό training set, χρησιμοποιώντας την  `train_test_split` του `scikit-learn` (σχολιασμένη γραμμή παρακάτω).

In [6]:
y_train = df_train[['PAX']]

from sklearn.model_selection import train_test_split
#df_train, df_test, y_train, y_test = train_test_split(df_train, y_train, test_size=0.2, random_state=42)

df_train.drop(df_train.columns[[12]], axis=1, inplace=True)

df_train.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,Distance
0,2011-12-05,ORD,Chicago,41.978603,-87.904842,EWR,Newark,40.692500,-74.168667,8.352941,5.667243,1527.939887
1,2013-03-01,ATL,Atlanta,33.636719,-84.428067,LGA,New York,40.777245,-73.872608,10.421053,10.001754,1181.336943
2,2012-02-06,ORD,Chicago,41.978603,-87.904842,BOS,Boston,42.364347,-71.005181,9.250000,7.136821,1879.752387
3,2012-12-10,ORD,Chicago,41.978603,-87.904842,SFO,San Francisco,37.618972,-122.374889,8.666667,7.404291,3833.459735
4,2012-04-13,SFO,San Francisco,37.618972,-122.374889,JFK,New York,40.639751,-73.778925,14.037037,9.858544,5403.554833


Για να μπορεί να δουλέψει ένα αλγόριθμος ταξινόμησης είναι απαραίτητο το training και το test set να έχουν ακριβώς τον ίδιο αριθμό στηλών (ίδιες μεταβλητές). Συνεπώς, πρέπει και στο test set να διαγράψουμε όλες τις στήλες εκτός από τις Departure και Arrival. Αυτό γίνεται με τον παρακάτω κώδικα.

In [7]:
#df_test.drop(df_test.columns[[3,4,6,7,8,10]], axis=1, inplace=True)

df_test.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,Distance
0,2012-10-21,DFW,Dallas-Fort Worth,32.896828,-97.037997,SFO,San Francisco,37.618972,-122.374889,14.600000,11.575837,2821.532925
1,2012-09-13,LAX,Los Angeles,33.942536,-118.408075,ATL,Atlanta,33.636719,-84.428067,14.730769,13.364304,3779.583120
2,2012-09-04,ORD,Chicago,41.978603,-87.904842,IAH,Houston,29.984433,-95.341442,8.470588,5.885551,823.505267
3,2012-08-13,DEN,Denver,39.861656,-104.673178,PHX,Phoenix,33.434278,-112.011583,8.200000,6.292853,845.501646
4,2012-09-10,ORD,Chicago,41.978603,-87.904842,SEA,Seattle,47.449000,-122.309306,12.090909,9.138662,3825.798893


Οι στήλες των training και test set περιέχουν κατηγορικές μεταβλητές των οποίων οι τιμές είναι αλφαριθμητικά. Οι αλγόριθμοι ταξινόμησης ωστόσο δουλεύουν μόνο με αριθμητικές τιμές. Χρησιμοποιούμε το αντικείμενο `LabelEncoder` του `scikit-learn` για να μετατρέψουμε τα αλφαριθμητικά σε αριθμητικές τιμές.

In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df_train['Departure'])
df_train['Departure'] = le.transform(df_train['Departure'])
df_train['Arrival'] = le.transform(df_train['Arrival'])
df_test['Departure'] = le.transform(df_test['Departure'])
df_test['Arrival'] = le.transform(df_test['Arrival'])
le.fit(df_train['CityDeparture'])
df_train['CityDeparture'] = le.transform(df_train['CityDeparture'])
#df_train['CityArrival'] = le.transform(df_train['CityArrival'])
df_test['CityDeparture'] = le.transform(df_test['CityDeparture'])
#df_test['CityArrival'] = le.transform(df_test['CityArrival'])
le = LabelEncoder()
le.fit(df_train['DateOfDeparture'])
df_train['DateOfDeparture'] = le.transform(df_train['DateOfDeparture'])
df_test['DateOfDeparture'] = le.transform(df_test['DateOfDeparture'])

Έπειτα, εκπαιδεύουμε έναν ταξινομητή logistic regression για να προβλέψουμε τις κατηγορίες αριθμού επιβατών των δεδομένων ελέγχου. Επιπλέον, αποθηκεύουμε τις προβλέψεις μας στο αρχείο `y_pred.txt` στο δίσκο.

In [15]:
#from sklearn.linear_model import LogisticRegression
import numpy as np

#y_train = df_train[['PAX']]

#from sklearn.model_selection import train_test_split
#df_train, df_test, y_train, y_test = train_test_split(df_train, y_train, test_size=0.2, random_state=42)

#df_train.drop(df_train.columns[[12]], axis=1, inplace=True)

X_train = df_train
X_test = df_test

y_train = np.ravel(y_train)

from sklearn import svm
clf = svm.SVC(gamma='auto')
#clf.fit(X_train, y_train) 

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

scores = cross_val_score(clf, X_train, y_train, cv=10)
scores

#clf = LogisticRegression()
#clf.fit(X_train, y_train)


ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [ ]:
y_pred = clf.predict(X_test)
#y_pred.shape

In [ ]:
import csv
with open('y_predict.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Id', 'Label'])
    for i in range(y_pred.shape[0]):
        writer.writerow([i, y_pred[i]])

In [ ]:
from sklearn.metrics import f1_score
#y_test = np.loadtxt('test_labels.csv', delimiter=",", skiprows=1, usecols=[1]) # test_labels.csv το χουν αυτοι? 
f1_score(y_test, y_pred, average='micro')

Υποβάλλουμε το αρχείο `y_pred.csv` στην πλατφόρμα και μας δίνει micro F1-score ίσο με 0.23.

Ένας εναλλακτικός τρόπος αναπαράστασης κατηγορικών μεταβλητών είναι το λεγόμενο one-hot encoding όπου υπάρχει διαθέσιμη μια μεταβλητή για κάθε πιθανή τιμή του χαρακτηριστικού και ανάλογα με την τρέχουσα τιμή του, μια από αυτές της μεταβλητές είναι 1, ενώ όλες οι άλλες παραμένουν 0. Για παράδειγμα, αν είχαμε κάποια μεταβλητή Weekday η οποία περιέγραφε τη μέρα που έγινε μια πτήση, θα είχαμε 7 μεταβλητές (π.χ. 1000000 για Monday, 0100000 για Tuesday κτλ.). Σημειώστε ότι με την one-hot encoding αναπαράσταση ο αριθμός των χαρακτηριστικών που προκύπτει είναι ίσος με τον αριθμό των διαφορετικών τιμών που παίρνει η μεταβλητή. Παρακάτω εφαρμόζουμε one-hot encoding στις μεταβλητές `Departure` και `Arrival`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cf = [0,1,2,3]
enc = OneHotEncoder(categorical_features=cf,sparse=False)
enc.fit(df_train)
#enc.fit(df_test)
X_train = enc.transform(df_train)
X_test = enc.transform(df_test)
X_train.shape

In [ ]:
print(X_train.shape[0])
print(X_train.shape[1])

In [ ]:
#PCA
from sklearn.decomposition import PCA
samples = X_train.shape[0]
dim = X_train.shape[1]
n_components=10
pca = PCA(n_components)
Xtr_reduced = pca.fit_transform(X_train)
Xte_reduced = pca.fit_transform(X_test)

Βλέπουμε ότι ο αριθμός των στηλών αυξήθηκε από 2 σε 40. Αυτό συνέβη γιατί υπάρχουν 20 διαφορετικά αεροδρόμια και συνεπώς χρειαζόμαστε 20 μεταβλητές για να αναπαραστήσουμε κάθε μια από τις μεταβλητές `Departure` και `Arrival` χρησιμοποιώντας one-hot encoding.

Έπειτα, εκπαιδεύουμε ξανά έναν ταξινομητή logistic regression για να προβλέψουμε τις κατηγορίες αριθμού επιβατών των δεδομένων ελέγχου. Επιπλέον, αποθηκεύουμε τις προβλέψεις μας στο αρχείο `y_pred.csv` στο δίσκο.

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
with open('y_pred.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Id', 'Label'])
    for i in range(y_pred.shape[0]):
        writer.writerow([i, y_pred[i]])

In [ ]:
f1_score(y_test, y_pred, average='micro')

Υποβάλλουμε το αρχείο `y_pred.csv` στην πλατφόρμα και μας δίνει macro F1-score ίσο με 0.33. Στα πλαίσια της παρούσας εργασίας καλείστε να τροποποιήσετε τον παραπάνω κώδικα ώστε να προβλέψετε τις κατηγορίες αριθμού επιβατών των πτήσεων του test set. Μπορείτε να εφαρμόσετε κάποια μέθοδο επιλογής χαρακτηριστικών στα δεδομένα ώστε να κρατήσετε
μόνο ένα υποσύνολο από τα χαρακτηριστικά. Μπορείτε επίσης να δημιουργήσετε νέα χαρακτηριστικά τα οποία
ίσως βοηθήσουν στην κατηγοριοποίηση. Μπορείτε επιπλέον να πειραματιστείτε με κάποια μέθοδο μείωσης
διάστασης και να διερευνήσετε αν η εφαρμογή της βελτιώνει το αποτέλεσμα της κατηγοριοποίησης. Επίσης,
μπορείτε να χρησιμοποιήσετε θορυβώδη ή ανούσια χαρακτηριστικά για να παράγετε νέα χαρακτηριστικά που
παρέχουν μεγαλύτερα ποσοστά πληροφορίας. Μπορείτε να χρησιμοποιήσετε διαφορετικούς ταξινομητές ή να συνδυάσετε τα αποτελέσματα περισσότερων από έναν ταξινομητές. 

### Παράδοση Εργασίας

Η εργασία είναι είτε ατομική ή μπορεί να γίνει σε ομάδες το πολύ 3 ατόμων. Η τελική αξιολόγηση θα βασίζεται τόσο στο micro F1-score που θα επιτύχετε, όσο και στη συνολική προσέγγισή σας στο πρόβλημα. Στα πλαίσια της εργασίας, θα πρέπει να υποβληθούν τα εξής:
<ul>
    <li>Μια αναφορά 2 σελίδων, στην οποία θα πρέπει να περιγράψετε την προσέγγιση και τις μεθόδους που χρησιμοποιήσατε. Δεδομένου ότι πρόκειται για ένα πραγματικό πρόβλημα ταξινόμησης, μας ενδιαφέρει να γνωρίζουμε πώς αντιμετωπίσατε κάθε στάδιο του προβλήματος, π.χ. τι είδους αναπαράσταση δεδομένων χρησιμοποιήσατε, τι χαρακτηριστικά χρησιμοποιήσατε, εάν εφαρμόσατε τεχνικές μείωσης διάστασης, ποιούς αλγορίθμους ταξινόμησης δοκιμάσατε και γιατί, την απόδοση των μεθόδων σας (macro F1-score και χρόνο εκπαίδευσης), τυχόν προσεγγίσεις που τελικά δεν λειτούργησαν, αλλά
είναι ενδιαφέρον να παρουσιάσετε, και γενικά, ότι νομίζετε ότι είναι ενδιαφέρον να αναφερθεί.</li>
    <li>Ενα φάκελο με τον κώδικα της εφαρμογής σας.</li>
    <li>Εναλλακτικά μπορείτε να συνδυάσετε τα δυο παραπάνω σε ένα αρχείο Ipython Notebook.</li>
    <li>Δημιουργήστε ένα αρχείο .zip που περιέχει τον κώδικα και την αναφορά και υποβάλετέ τον στην πλατφόρμα e-class.</li>
    <li>**Λήξη προθεσμίας**: 6 Ιανουαρίου 2019.</li>
</ul>